<a href="https://colab.research.google.com/github/ronierc/PythonIA/blob/main/ConsultandoEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 7.6 MB/s eta 0:00:00


In [2]:
#Importações e Configurações Iniciais
import numpy as np #Manipulação de Vetores [Cálculos matemáticos]
import pandas as pd #Manipular DataFrames [Google Sheets/Excel - Tabelas]
import google.generativeai as genai #Importa a SDK da Genai e apelida

from google.colab import userdata #Biblioteca do Secret
GOOGLE_API_KEY = userdata.get('SECRET_KEY') #Resgatando chave

genai.configure(api_key=GOOGLE_API_KEY) #Setando na configuração

Embedding: São formas de representar Tokens em vetores [Correlaciona palavras. Exemplo "Preciso extrair os 'Dados' do banco." ele correlaciona o 'Dados' com banco pra não achar que são dados de brinquedo.]
Sempre retorna em 768 posições.


In [4]:
#Listar os modelos de embedding disponiveis
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
#Exemplo de Embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
"\n"
"Artigo completo:\n"
"\n")

embeddings = genai.embed_content(model="models/embedding-001", #necessário colocar o models na frente desse
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT" #Tipo do cenário que estou trabalhando [https://ai.google.dev/gemini-api/tutorials/document_search?hl=pt-br]
                                 )

print(embeddings)

{'embedding': [0.0553135, 0.0077083623, -0.0354181, -0.022514744, 0.08151982, 0.013521809, -0.027504247, -0.00086904416, 0.04278047, 0.060694084, 0.002546985, 0.044232614, -0.036569327, 0.0018963555, 0.016158564, -0.004908304, 0.03143286, -0.026755683, -0.02063205, -0.00698819, 0.016526327, 0.0060043395, -0.04357717, -0.047922496, -0.02842969, 0.011861899, -0.018210717, -0.045655057, 0.0006590934, 0.0186698, -0.045655765, 0.04363548, -0.016757434, 0.022653304, -0.048464227, 0.008755076, -0.05633565, -0.01850483, -0.004039444, 0.0011850819, -0.025212279, -0.077321224, -0.013141193, 0.053732064, 0.022353856, -0.039701045, 0.036702618, 0.01208116, 0.008602891, -0.048954852, -0.0012578157, 0.01507516, 0.068693936, -0.018314548, -0.004173154, 0.0026827783, -0.0056239185, -0.035728164, 0.017272122, -0.018349621, 0.0036099204, 0.013493346, -0.014871403, 0.044343006, 0.012889104, -0.04559449, -0.033156924, -0.0034231108, -0.0010768645, 0.0705541, -0.0006658671, -0.010847085, 0.042600058, -0.06

In [6]:
DOCUMENT1 = {
    "Título": "Tesla Phone: Uma Revolução na Integração Veicular:",
    "Conteúdo": "O Tesla Phone promete ser uma extensão perfeita do seu Tesla, oferecendo controle intuitivo do carro, desempenho otimizado para tarefas relacionadas ao veículo e abrindo portas para um futuro de integração com outros produtos Tesla e um sistema doméstico inteligente completo."
}
DOCUMENT2 = {
    "Título": "Além de um Smartphone: O Tesla Phone como Centro de Comando Pessoal:",
    "Conteúdo": "O Tesla Phone vai além das funções básicas de um smartphone, agindo como um assistente pessoal inteligente, gerenciando sua agenda, fazendo reservas, controlando dispositivos domésticos e te conectando ao metaverso. Além disso, oferece segurança aprimorada com autenticação biométrica robusta e um portal para realidades virtuais e aumentadas."
}
DOCUMENT3 = {
    "Título": "Sustentável e Ético: O Tesla Phone Reinventando a Tecnologia Móvel:",
    "Conteúdo": "O Tesla Phone se destaca pelo compromisso com a sustentabilidade, utilizando materiais reciclados e produção ética com condições de trabalho justas. Projetado para durar e ser facilmente reciclado, o dispositivo busca minimizar o impacto ambiental da tecnologia móvel."
}

documents = [DOCUMENT1,DOCUMENT2,DOCUMENT3]

In [9]:
df = pd.DataFrame(documents) #Converte em DataFrame [transforma em uma planilha]
df.columns = ["Titulo", "Conteudo"] #nomea as colunas como quiser
df

,Titulo,Conteudo
0,Tesla Phone: Uma Revolução na Integração Veicu...,O Tesla Phone promete ser uma extensão perfeit...
1,Além de um Smartphone: O Tesla Phone como Cent...,O Tesla Phone vai além das funções básicas de ...
2,Sustentável e Ético: O Tesla Phone Reinventand...,O Tesla Phone se destaca pelo compromisso com ...


In [10]:
model="models/embedding-001"

In [12]:
def embed_fn(title, text): #Função de entrada
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT"
                                 ) ["embedding"] #Vai ser armazenado toda a informação. TEM QUE SER ESSE NOME

In [15]:
#Crias uma coluna
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]), axis=1) #aplica na extrutura
df

,Titulo,Conteudo,Embeddings
0,Tesla Phone: Uma Revolução na Integração Veicu...,O Tesla Phone promete ser uma extensão perfeit...,"[0.025719099, -0.022814443, -0.0073978463, 0.0..."
1,Além de um Smartphone: O Tesla Phone como Cent...,O Tesla Phone vai além das funções básicas de ...,"[0.031571746, -0.044519193, -0.029418135, 0.02..."
2,Sustentável e Ético: O Tesla Phone Reinventand...,O Tesla Phone se destaca pelo compromisso com ...,"[0.011760247, -0.023999857, -0.026862994, 0.03..."


In [17]:
#Cria uma varíavel para capturar a pergunta
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta, #Executa a consulta no meu embed
                                 task_type="RETRIEVAL_QUERY" #Mudou nesse exemplo agora é uma consulta
                                 )

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta["embedding"]) #valida a consulta no embedding

  indice = np.argmax(produtos_escalares) #aqui onde ele busca o com maior singularidade

  return df.iloc[indice]["Conteudo"]

Modelo abaixo vai trazer o Conteúdo exato imposto no documeto.

In [18]:
consulta = "O telefone controla dispositivos domésticos?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

O Tesla Phone vai além das funções básicas de um smartphone, agindo como um assistente pessoal inteligente, gerenciando sua agenda, fazendo reservas, controlando dispositivos domésticos e te conectando ao metaverso. Além disso, oferece segurança aprimorada com autenticação biométrica robusta e um portal para realidades virtuais e aumentadas.


Modelo abaixo vai trazer o Conteúdo reescrito pela IA.

In [25]:
generation_config = {
    "candidate_count":1,
    "temperature": 0.5
}

prompt = f"Reescreva esse texto de uma forma mais descontraida, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel(model_name="gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt) #Gera o conteudo da consulta
print(response.text)

O Tesla Phone é tipo o seu parça tecnológico supremo!

Ele não é só um celular, é um assistente pessoal que te ajuda com tudo: agenda, reservas, controle da casa e até te leva pro metaverso.

E tem mais! Ele é seguro pra caramba, com reconhecimento facial e impressão digital. E ainda te conecta a mundos virtuais e aumentados, pra você dar um rolê incrível!
